In [1]:
This notebook contains my replication of [this](https://jakevdp.github.io/blog/2014/06/10/is-seattle-really-seeing-an-uptick-in-cycling/) blog post by [Jake VanderPlan](http://vanderplas.com/) on using data from bicycle traffic across Seattle's Fremont Bridge to learn about commuting patterns.

Next steps:
- Follow [supervised learning]
- Follow [unsupervised learning]

SyntaxError: invalid syntax (<ipython-input-1-c2fcb41a5114>, line 1)